In [1]:
import numpy as np
import time
from skimage.morphology import label
import requests
from bs4 import BeautifulSoup

In [2]:
# Going back to look at an old puzzle to try to get started on the Dec 2020
# Use backtracking to solve


url='https://www.janestreet.com/puzzles/twenty-four-seven/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
x =[text for text in soup.body.stripped_strings]

print("".join(x[7:]))

The grid is incomplete. Place numbers in some of the empty cells below so that in total the grid contains one 1, two 2’s, etc., up to seven 7’s. Furthermore, each row and column must contain exactly 4 numbers which sum to 20. Finally, the numbered cells must form a connected region*, but every 2-by-2 subsquare in the completed grid must contain at least one empty cell.The answer to this puzzle is the product of the areas of the connected groups of empty squares in the completed grid.(*Updated 2018-06-14 to clarify that cells connect alongedges only.)


<img src="https://www.janestreet.com/puzzles/wp-content/uploads/2018/06/20180604_20_4_7.png" width="250" height="300">

In [3]:
grid =np.array([[0,4,0,0,0,0,0],
                [0,0,6,3,0,0,6],
                [0,0,0,0,0,5,5],
                [0,0,0,4,0,0,0],
                [4,7,0,0,0,0,0],
                [2,0,0,7,4,0,0],
                [0,0,0,0,0,1,0]])


grid[grid==0]=-1

In [4]:
#Forming the Puzzle Grid
class Matrix():
    def __init__(self,puzzle_string):
        self.grid = puzzle_string
        self.counts = {i:i-np.sum(self.grid==i) for i in range(8)}
        print('The problem - Start {}'.format(time.strftime('%X')))
        self.printGrid()
    
#Function to print the grid
    def printGrid(self):
        print(self.grid)
        
#Function to check if a digit can be placed in the given block
#Various checks
    def possible(self,row,col,digit):

        # Check row and columns count
        if digit ==0:
            if np.sum(self.grid[:,col]==0) > 2:
                return False
            if np.sum(self.grid[row,:]==0) > 2:
                return False
            
        # Check that final number completes
            if np.sum(self.grid[row,:]==-1) ==1:
                 if np.sum(self.grid[row,:])+digit+1  != 20:
                        return False
                    
            if np.sum(self.grid[:,col]==-1) ==1:
                 if np.sum(self.grid[:,col])+digit+1  != 20:
                        return False
        
        # Check row and columns sum
        if np.sum(self.grid[:,col])+digit+np.sum(self.grid[:,col]==-1) > 20:
            return False
        
        if np.sum(self.grid[row,:])+digit+np.sum(self.grid[row,:]==-1)  > 20:
            return False
        
        #check 2x2
        if digit !=0:
            if self.twobytwo(row,col):
                return False
            
        # Check Fobidden nums
            if self.more_constraints(self.grid[row,:],digit):
                return False
        
            if self.more_constraints(self.grid[:,col],digit):
                return False
            
        #checkconnected
        if np.max(label(self.grid!=0,connectivity=1)) >1 :
            return False
        
        # All tests pass return True
        return True
    
    def twobytwo(self,i,j):
        if (i > 0 and j > 0 and 
        self.grid[i-1,j-1] > 0 and 
        self.grid[i-1,j] > 0 and 
        self.grid[i,j-1] > 0):
            return True
    
        if (i > 0 and j < 6 and
        self.grid[i-1,j+1] > 0 and 
        self.grid[i-1,j] > 0 and 
        self.grid[i,j+1] > 0):
            return True
    
        if (i < 6 and j > 0 and
        self.grid[i+1,j-1] > 0 and 
        self.grid[i+1,j] > 0 and 
        self.grid[i,j-1] > 0):
            return True

    
        if (i < 6 and j < 6 and
        self.grid[i+1,j+1] > 0 and 
        self.grid[i+1,j] > 0 and 
        self.grid[i,j+1] > 0):
            return True
    
        return False
    
    def more_constraints(self,array,digit):
    
        if np.sum(array==1) > 0:
            if digit < 5:
                return True
            return False
        
        if np.sum(array==2) > 0:
            if digit < 4 :
                return True
            return False
    
        if np.sum(array==3) > 0:
            if digit < 3 :
                return True
            return False
    
        if np.sum(array==4) > 0:
            if digit < 2 :
                return True
            return False
        
        return False
        

    def solve(self):
        
        for row in range(7):
            for col in range(7):
                if self.grid[row][col] == -1:
                    for digit in range(7,-1,-1):
                        if (digit ==0 or self.counts[digit]>0):
                            if self.possible(row,col,digit):
                                self.grid[row,col] = digit
                                self.counts[digit] -= 1
                                self.solve()
                                self.grid[row,col] = -1  #Backtrack step
                                self.counts[digit] += 1
                    return False
        
        print('\nThe solution - End {}'.format(time.strftime('%X')))
        self.printGrid()

In [ ]:
x = Matrix(grid)
x.solve()
print('\n Finish {}'.format(time.strftime('%X')))

The problem - Start 09:31:55
[[-1  4 -1 -1 -1 -1 -1]
 [-1 -1  6  3 -1 -1  6]
 [-1 -1 -1 -1 -1  5  5]
 [-1 -1 -1  4 -1 -1 -1]
 [ 4  7 -1 -1 -1 -1 -1]
 [ 2 -1 -1  7  4 -1 -1]
 [-1 -1 -1 -1 -1  1 -1]]

The solution - End 09:32:24
[[7 4 3 0 6 0 0]
 [0 0 6 3 5 0 6]
 [0 0 5 0 5 5 5]
 [0 3 6 4 0 0 7]
 [4 7 0 0 0 7 2]
 [2 0 0 7 4 7 0]
 [7 6 0 6 0 1 0]]


### Solution is :

<img src="https://www.janestreet.com/puzzles/wp-content/uploads/2018/07/20180702_20_4_7_ans.png" width="250" height="300">